In [37]:
import os

In [38]:
import os
import re
import pandas as pd

# 루트 경로 설정
root_path = "/workspace/results/condition_explore"
csv_path = f"{root_path}/summary_results.csv"
setting_file_name = "_settings.txt"
results_file_name = "_avg_results.txt"

# 데이터 저장용 리스트
data_list = []

# 폴더 리스트 가져오기
folder_list = os.listdir(root_path)

for folder in folder_list:
    result_path = os.path.join(root_path, folder, results_file_name)
    setting_path = os.path.join(root_path, folder, setting_file_name)

    # 결과 파일과 설정 파일이 모두 존재하지 않으면 건너뛰기
    if not os.path.exists(result_path) or not os.path.exists(setting_path):
        continue

    # 결과 파일 읽기
    with open(result_path, "r") as file:
        result_text = file.read()

    # PESQ, ESTOI, SI-SDR 추출 (± 앞 숫자만 가져옴)
    pesq_match = re.search(r"PESQ:\s*([-]?\d+\.\d+)", result_text)
    estoi_match = re.search(r"ESTOI:\s*([-]?\d+\.\d+)", result_text)
    si_sdr_match = re.search(r"SI-SDR:\s*([-]?\d+\.\d+)", result_text)

    pesq = float(pesq_match.group(1)) if pesq_match else None
    estoi = float(estoi_match.group(1)) if estoi_match else None
    si_sdr = float(si_sdr_match.group(1)) if si_sdr_match else None

    # 설정 파일 읽기
    with open(setting_path, "r") as file:
        setting_text = file.read()

    # epoch, mode, evaluationnumbers, data 추출
    epoch_match = re.search(r"epoch:\s*(\d+)", setting_text)
    mode_match = re.search(r"mode:\s*(\w+)", setting_text)
    eval_match = re.search(r"evaluationnumbers:\s*(\d+)", setting_text)
    data_match = re.search(r"data:\s*(.+)", setting_text)

    epoch = int(epoch_match.group(1)) if epoch_match else None
    mode = mode_match.group(1) if mode_match else None
    evaluationnumbers = int(eval_match.group(1)) if eval_match else None
    data = os.path.basename(data_match.group(1)) if data_match else None

    # 리스트에 추가
    data_list.append([folder, pesq, estoi, si_sdr, epoch, mode, evaluationnumbers, data])

# DataFrame 생성
df = pd.DataFrame(data_list, columns=["folder", "PESQ", "ESTOI", "SI-SDR", "epoch", "mode", "evaluationnumbers", "data"])

# CSV 저장
df.to_csv(csv_path, index=False)

# CSV 파일 표시
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Summary Results", dataframe=df)


In [39]:
import os
import re
import pandas as pd

# 경로 설정
root_path = "/workspace/results/condition_explore"
csv_path = f"{root_path}/summary_combined_results.csv"
results_file_name = "_avg_results.txt"
setting_file_name = "_settings.txt"

# 데이터 저장 리스트
data_list = []

# 폴더 순회
folder_list = os.listdir(root_path)

for folder in folder_list:
    result_path = os.path.join(root_path, folder, results_file_name)
    setting_path = os.path.join(root_path, folder, setting_file_name)

    # 파일 존재 여부 확인
    if not os.path.exists(result_path) or not os.path.exists(setting_path):
        continue

    # 결과 파일 읽기
    with open(result_path, "r") as f:
        result_text = f.read()

    # 정규표현식으로 성능 지표 추출
    pesq = re.search(r"PESQ:\s*([-]?\d+\.\d+)", result_text)
    estoi = re.search(r"ESTOI:\s*([-]?\d+\.\d+)", result_text)
    si_sdr = re.search(r"SI-SDR:\s*([-]?\d+\.\d+)", result_text)
    si_sar = re.search(r"SI-SAR:\s*([-]?\d+\.\d+)", result_text)
    si_sir = re.search(r"SI-SIR:\s*([-]?\d+\.\d+)", result_text)

    # 존재 여부 확인 후 float 변환
    pesq_val = float(pesq.group(1)) if pesq else None
    estoi_val = float(estoi.group(1)) if estoi else None
    si_sdr_val = float(si_sdr.group(1)) if si_sdr else None
    si_sar_val = float(si_sar.group(1)) if si_sar else None
    si_sir_val = float(si_sir.group(1)) if si_sir else None

    # 설정 파일 읽기
    with open(setting_path, "r") as f:
        setting_text = f.read()

    mode_match = re.search(r"mode:\s*(\S+)", setting_text)
    data_match = re.search(r"data:\s*(.+)", setting_text)
    n_match = re.search(r"N:\s*(\d+)", setting_text)

    mode = mode_match.group(1) if mode_match else None
    dataset = os.path.basename(data_match.group(1)) if data_match else None
    N = int(n_match.group(1)) if n_match else None

    # 결과 리스트에 추가
    data_list.append([folder, pesq_val, estoi_val, si_sdr_val, si_sar_val, si_sir_val, N, dataset, mode])

# 데이터프레임 생성
df = pd.DataFrame(data_list, columns=["folder", "PESQ", "ESTOI", "SI-SDR", "SI-SAR", "SI-SIR", "N", "dataset", "mode"])

# CSV로 저장
df.to_csv(csv_path, index=False)
